In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#pip install git@github.com:badw/SMILESBOX.git
#pip install git@github.com:badw/reactit.git
#pip install mace-torch

## Generating "backend" data via 

1. ML inter-atomic potentials such as MACE (https://github.com/ACEsuit/mace)[https://github.com/ACEsuit/mace]

2. DFT using VASP (https://vasp.at)[https://vasp.at]

3. coupled cluster using PSI4 (https://psicode.org)[https://psicode.org]

In [2]:
from ase.optimize import QuasiNewton
from ase.vibrations import Vibrations
from smilesbox.smilesbox import SMILESbox
from arcs.generate import GetEnergyandVibrationsAseCalc,GetEnergyandVibrationsVASP, GetEnergyandVibrationsPsi4
import shutil
from mace.calculators import MACECalculator 
from mace.calculators import mace_off

/Users/badw/miniconda3/envs/py312/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  _Jd, _W3j_flat, _W3j_indices = torch.load(os.path.join(os.path.dirname(__file__), 'constants.pt'))


cuequivariance or cuequivariance_torch is not available. Cuequivariance acceleration will be disabled.


we can set up a reaction graph between the following reagents:

1. H<sub>2</sub>
2. O<sub>2</sub>
3. H<sub>2</sub>O
4. H<sub>2</sub>O<sub>2</sub>

In [3]:
from reactit.reactit import ReactionGenerator

# define a dictionary with key: formula, value: SMILES
reagents = {'H2':'[HH]','O2':'O=O','H2O':'O','H2O2':'OO'}

# generate all possible reactions between these reagents
rg = ReactionGenerator(compounds=list(reagents))

reactions = rg.iterate()
reactions

  0%|          | 0/11 [00:00<?, ?it/s]

['2 H2 + 1 O2 = 2 H2O',
 '1 O2 + 2 H2O = 2 H2O2',
 '1 H2 + 1 O2 = 1 H2O2',
 '1 H2 + 1 H2O2 = 2 H2O']

## 1. using MACE

In [10]:
%%capture 
calc = mace_off(model='large')
try:
    shutil.rmtree('./vib/')
except FileNotFoundError:
    pass

dft_dict = {}
for reagent in reagents:
    sm = SMILESbox(smiles=reagents[reagent])
    
    atoms = sm.atoms
    atoms.calc = calc

    QuasiNewton(atoms).run(fmax=0.001)

    vib = Vibrations(atoms,nfree=2) 

    vib.run()
    gevac = GetEnergyandVibrationsAseCalc(aseatomscalc=atoms,asevibrationscalc=vib)

    dft_dict[reagent] = gevac.as_dict()
    
    #cleanup
    try:
        shutil.rmtree('./vib/')
    except FileNotFoundError:
        pass

In [11]:
dft_dict['H2']

{'atoms': {'numbers': array([1, 1]),
  'positions': array([[ 1.00836894e+00,  1.59239786e-03, -6.40909418e-02],
         [ 1.73412132e+00,  1.59239786e-03, -6.40909418e-02]]),
  'initial_magmoms': array([1., 1.]),
  'cell': array([[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]),
  'pbc': array([False, False, False])},
 'pointgroup': 'D*h',
 'spin': 0,
 'rotation_num': 2,
 'islinear': 'linear',
 'energy': -31.86735740812427,
 'vibrations': array([0.00000000e+00+4.21746591e-09j, 0.00000000e+00+4.75788557e-11j,
        1.99234341e-09+0.00000000e+00j, 4.84772929e-03+0.00000000e+00j,
        4.84772929e-03+0.00000000e+00j, 4.31490404e-01+0.00000000e+00j])}

In [20]:
from arcs.generate import GraphGenerator

dft_dict['reactions'] = rg.as_dict() # add to the dft_dict
graph = GraphGenerator().from_dict(dft_dict=dft_dict,temperature=298.15,pressure=1.0)

/Users/badw/miniconda3/envs/py312/lib/python3.12/site-packages/ase/thermochemistry.py:823: UserWarning: 1 imag modes removed
  warn(f"{n_imag} imag modes removed", UserWarning)


In [21]:
from arcs.traversal import Traversal
from arcs.generate import GenerateInitialConcentrations

gic = GenerateInitialConcentrations(graph=graph).update_ic(
    {'H2':10,'O2':10}
    )

t = Traversal(graph=graph)

data = t.sample(initial_concentrations=gic,ncpus=4,nsamples=1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [22]:
from arcs.analysis import AnalyseSampling
import pandas as pd 

analysis = AnalyseSampling()
stats = pd.Series(analysis.reaction_statistics(data)).sort_values(ascending=False)
stats.head(10)

2 H2 + 1 O2 = 2 H2O      1696
1 H2 + 1 O2 = 1 H2O2      662
1 O2 + 2 H2O = 2 H2O2     404
1 H2 + 1 H2O2 = 2 H2O     138
dtype: int64

In [23]:
average_data = pd.DataFrame(analysis.average_sampling(data))
average_data = average_data.loc[~(average_data==0).all(axis=1)]
average_data.sort_values(by='diff',inplace=True)
print(average_data.round(2).to_string())

      initial  mean   diff   sem   std    var
H2       10.0  0.00 -10.00  0.00  0.00   0.00
O2       10.0  3.77  -6.23  0.03  1.83   3.34
H2O2      0.0  2.47   2.47  0.07  3.65  13.35
H2O       0.0  7.53   7.53  0.07  3.65  13.35


## 2. DFT within VASP

In [ ]:
%%capture 

from ase.calculators.vasp import Vasp 
import os 
os.environ['VASP_PP_PATH'] = '/path/to/VASP_pseudopotentials/'
calc = Vasp(
    command='mpirun -np 4 /path/to/vasp/bin/vasp_gam',
            directory='./vasp_calculations/relax/',
            kpts=(1, 1, 1),
            xc='PBE',
            encut=400,
            ediff=1e-6,
            ediffg=-1e-4,
            ibrion=1,
            isym=2,
            symprec=0.01,
            isif=2,
            nsw=100,
            lreal=False,
            lwave=False,
            lcharg=False,
            algo='All',
            potim=0.1,
            ncore=4,
            prec='accurate',
            addgrid=True,
            gamma=True
)

vibcalc = Vasp(
    command='mpirun -np 4 /path/to/vasp/bin/vasp_gam',
            directory='./vasp_calculations/relax/',
            kpts=(1, 1, 1),
            xc='PBE',
            encut=400,
            ediff=1e-6,
            ediffg=-1e-4,
            ibrion=6,
            isym=2,
            symprec=0.01,
            isif=2,
            nsw=10,
            lreal=False,
            lwave=False,
            lcharg=False,
            algo='All',
            potim=0.1,
            ncore=4,
            prec='accurate',
            addgrid=True,
            gamma=True
)

dft_dict = {}
for reagent in reagents:

    if reagent == 'O2':
        calc.set(**{'ispin':2,'nupdown':2}) # triplet oxygen 
        vibcalc.set(**{'ispin':2,'nupdown':2})
    else:
        calc.set(**{'ispin':1,'nupdown':0})
        vibcalc.set(**{'ispin':1,'nupdown':0})

    sm = SMILESbox(smiles=reagents[reagent])
    sm.add_box([10,10,10]) 
    atoms = sm.atoms
    
    atoms.calc = calc

    atoms.get_potential_energy()
    try:
        vibcalc.calculate(atoms)
    except AttributeError:
        pass 

    gevac = GetEnergyandVibrationsVASP(relax_directory='./vasp_calculations/relax/',vibrations_directory='./vasp_calculations/vibrations/')

    dft_dict[reagent] = gevac.as_dict()

    #cleanup calculations 
    try:
        shutil.rmtree('./vib/')
        shutil.rmtree('./vasp_calculations/')
    except FileNotFoundError:
        pass

In [ ]:
from arcs.generate import GraphGenerator
dft_dict['reactions'] = rg.as_dict()
graph = GraphGenerator().from_dict(dft_dict=dft_dict,temperature=298.15,pressure=1.0)

In [ ]:
from arcs.traversal import Traversal
from arcs.generate import GenerateInitialConcentrations

gic = GenerateInitialConcentrations(graph=graph).update_ic(
    {'H2':10,'O2':10}
    )

t = Traversal(graph=graph)

data = t.sample(initial_concentrations=gic,ncpus=4,nsamples=1000)

In [ ]:
from arcs.analysis import AnalyseSampling
import pandas as pd 

analysis = AnalyseSampling()
stats = pd.Series(analysis.reaction_statistics(data)).sort_values(ascending=False)
stats.head(10)

In [ ]:
average_data = pd.DataFrame(analysis.average_sampling(data))
average_data = average_data.loc[~(average_data==0).all(axis=1)]
average_data.sort_values(by='diff',inplace=True)
print(average_data.round(2).to_string())

## 3. with PSI4

In [ ]:
#can be necessary if kernel crashes 
#import psi4 

#psi4.core.clean()

In [ ]:
%%capture 
from ase.calculators.psi4 import Psi4
import numpy as np
import os

inversecmtoev = 1.23981e-4

#cleanup
try:
    shutil.rmtree('./vib/')
    shutil.rmtree('./psi4_calculation')
except FileNotFoundError:
    pass

basis = 'cc-pvdz'
#basis = 'sto-3g'
method = 'ccsd'

dft_dict = {}
for reagent,smiles in reagents.items():

    sm = SMILESbox(smiles=reagents[reagent])     
    atoms = sm.atoms


    calc = Psi4(
        atoms=atoms,
            method = method,
            memory = '1 GB', 
            basis = basis,
            directory='psi4_calculation',
            ) 
    
    calc.psi4.core.clean()
    calc.psi4.set_num_threads(8)
    calc.psi4.core.set_global_option('cachelevel',2) #set to 0 if problems exist
    
    atoms.calc = calc    
    
    e = calc.psi4.optimize(method+'/'+basis,molecule=calc.molecule,return_wfn=False)
    calc.molecule.set_geometry(calc.molecule.geometry())
    _,w = calc.psi4.frequencies(method+'/'+basis,molecule=calc.molecule,return_wfn=True)
    
    gevac = GetEnergyandVibrationsPsi4(aseatoms=atoms,energy=e,vibrations=w.frequency_analysis['omega'].data)    

    _dict = gevac.as_dict()
    dft_dict[reagent] = _dict
    #cleanup 
    try:
        shutil.rmtree('./vib/')
        shutil.rmtree('./psi4_calculation')
    except FileNotFoundError:
        pass

H2

  Memory set to 953.674 MiB by Python driver.
  Threads set to 1 by Python driver.
  Threads set to 8 by Python driver.

Scratch directory: /tmp/
gradient() will perform analytic gradient computation.
   => Libint2 <=

    Primary   basis highest AM E, G, H:  6, 6, 3
    Auxiliary basis highest AM E, G, H:  7, 7, 4
    Onebody   basis highest AM E, G, H:  -, -, -
    Solid Harmonics ordering:            Gaussian

*** tstart() called on dhcp-10-24-7-132.wlan.ntnu.no
*** at Mon May 19 15:47:49 2025

   => Loading Basis Set <=

    Name: CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1-2 entry H          line    22 file /Users/badw/miniconda3/envs/py312/share/psi4/basis/cc-pvdz.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        8 Threads,   

In [6]:
dft_dict['H2O2']

{'atoms': {'numbers': array([ 8,  8,  1,  1]),
  'positions': array([[ 1.02370645,  0.01647603,  0.10752139],
         [ 2.47186171,  0.00686660, -0.01461852],
         [ 0.83058277, -0.05857518, -0.84641164],
         [ 2.66498539,  0.08191780,  0.93931452]]),
  'initial_magmoms': array([ 0.00000000,  0.00000000,  1.00000000,  1.00000000]),
  'cell': array([[ 0.00000000,  0.00000000,  0.00000000],
         [ 0.00000000,  0.00000000,  0.00000000],
         [ 0.00000000,  0.00000000,  0.00000000]]),
  'pbc': array([False, False, False])},
 'pointgroup': 'C2h',
 'spin': 0,
 'rotation_num': 2,
 'islinear': 'nonlinear',
 'energy': -4114.016523830005,
 'vibrations': array([ 0.00000000+0.03268008j,  0.00000000+0.00000000j,
         0.00000000+0.00000000j,  0.00000000+0.00000000j,
         0.00000000+0.00000000j,  0.00000001+0.00000000j,
         0.00000001+0.00000000j,  0.11418737+0.00000000j,
         0.15492548+0.00000000j,  0.19194415+0.00000000j,
         0.47637336+0.00000000j,  0.47722

In [15]:
from arcs.generate import GraphGenerator
dft_dict['reactions'] = rg.as_dict()
graph = GraphGenerator().from_dict(dft_dict=dft_dict,temperature=298.15,pressure=1.0)

/Users/badw/miniconda3/envs/py312/lib/python3.12/site-packages/ase/thermochemistry.py:823: UserWarning: 1 imag modes removed
  warn(f"{n_imag} imag modes removed", UserWarning)


In [16]:
from arcs.traversal import Traversal
from arcs.generate import GenerateInitialConcentrations

gic = GenerateInitialConcentrations(graph=graph).update_ic(
    {'H2':10,'O2':10}
    )

t = Traversal(graph=graph)

data = t.sample(initial_concentrations=gic,ncpus=4,nsamples=1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [17]:
from arcs.analysis import AnalyseSampling
import pandas as pd 

analysis = AnalyseSampling()
stats = pd.Series(analysis.reaction_statistics(data)).sort_values(ascending=False)
stats.head(10)

2 H2 + 1 O2 = 2 H2O      1867
1 H2 + 1 O2 = 1 H2O2      571
1 O2 + 2 H2O = 2 H2O2     427
1 H2 + 1 H2O2 = 2 H2O     123
dtype: int64

In [18]:
average_data = pd.DataFrame(analysis.average_sampling(data))
average_data = average_data.loc[~(average_data==0).all(axis=1)]
average_data.sort_values(by='diff',inplace=True)
print(average_data.round(2).to_string())

      initial  mean   diff   sem   std    var
H2       10.0  0.00 -10.00  0.00  0.00   0.00
O2       10.0  4.04  -5.96  0.03  1.83   3.34
H2O2      0.0  1.91   1.91  0.07  3.65  13.35
H2O       0.0  8.09   8.09  0.07  3.65  13.35


plot an interactive graph 

In [ ]:
pyvis_kwargs = {'width':'50%','notebook':False,"font_color":'white','directed':True}
g = analysis.result_to_pyvis(data,head=20,**pyvis_kwargs)
g.save_graph(name="example_pyvis_graph.html")

In [ ]:
! open example_pyvis_graph.html 